In [ ]:
# This code is prepared by Orhan Erdem
# Please email orhanerdem at gmail.com for errors, suggestions

In [ ]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt
from sklearn import metrics

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

%matplotlib inline

In [ ]:
#Load data
asses_df=pd.read_excel('2023_Roxbury.xlsx')
asses_df=asses_df.dropna()
asses_df.head()

In [ ]:
asses_df.describe()

In [ ]:
#create a list of predictor variables
#create a list of predicto variables
excludeColumns=('TOTAL_VALUE','LU','HEAT_TYPE','PROP_VIEW')
predictors=[s for s in asses_df.columns if s not in excludeColumns]
outcome='TOTAL_VALUE'

X=asses_df[predictors]
y=asses_df[outcome]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# train linear regression model
reg = LinearRegression()
reg.fit(train_X, train_y)

In [ ]:
print('intercept',reg.intercept_)
print(pd.DataFrame({'Predictor':X.columns,'coefficient':reg.coef_}))

#Prediction
y_valid_pred=reg.predict(valid_X)
y_train_pred=reg.predict(train_X)

# Training Part Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(train_y, y_train_pred)
rmse=mean_squared_error(train_y, y_train_pred,squared=False)
print("Mean Squared Error:", mse.round(2))
print("Root Mean Squared Error:", rmse.round(2))

Mean Squared Error: 6100468727.14
Root Mean Squared Error: 78105.5


In [ ]:
regressionSummary(train_y,reg.predict(train_X))


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 78105.4974
            Mean Absolute Error (MAE) : 27861.2601
          Mean Percentage Error (MPE) : -1.6172
Mean Absolute Percentage Error (MAPE) : 5.7576


# Validation Part Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(valid_y, y_valid_pred)
rmse=mean_squared_error(valid_y, y_valid_pred,squared=False)
print("Mean Squared Error:", mse.round(2))
print("Root Mean Squared Error:", rmse.round(2))

Mean Squared Error: 3583622101.87
Root Mean Squared Error: 59863.36


In [ ]:
regressionSummary(valid_y,reg.predict(valid_X))


Regression statistics

                      Mean Error (ME) : -918.7462
       Root Mean Squared Error (RMSE) : 59863.3619
            Mean Absolute Error (MAE) : 28309.8071
          Mean Percentage Error (MPE) : -1.4364
Mean Absolute Percentage Error (MAPE) : 6.1946


# R2 Akaike Information Criterion (AIC) and Bayesian Information Criterion(BIC)

In [ ]:
#Calcualte R2
from sklearn.metrics import r2_score

r2=r2_score(train_y, y_train_pred)
p=len(predictors)
n=len(train_y)
adjusted_r2=1-(1-r2)*(n-1)/(n-p-1)

print("R2:",r2.round(2))
print("Adjusted R2", adjusted_r2.round(2))

R2: 0.89
Adjusted R2 0.89


In [ ]:
print('adjusted r2',adjusted_r2_score(train_y,y_train_pred,reg).round(2))
print('AIC',AIC_score(train_y, y_train_pred,reg).round(2))
print('BIC',BIC_score(train_y, y_train_pred,reg).round(2))

adjusted r2 0.89
AIC 54547.07
BIC 54626.49


In [ ]:
#Use predict() to make prediction on a new set (validation set, for example)
reg_pred=reg.predict(valid_X)

result=pd.DataFrame({'Predicted':reg_pred,'Actual': valid_y,
                    'Residual':valid_y-reg_pred})

print(result.head(20))

#Compute also common accuracy metrics
regressionSummary(valid_y,reg_pred)

          Predicted  Actual      Residual
805   869505.942030  929700  60194.057970
5766  457763.019923  457600   -163.019923
189   405455.534288  393100 -12355.534288
6038  362853.182817  335400 -27453.182817
4582  680656.337252  671300  -9356.337252
1157  489275.379171  495000   5724.620829
4910  611823.974828  621000   9176.025172
5036  609971.743041  602800  -7171.743041
4049  774753.755597  772200  -2553.755597
4683  682267.191486  661900 -20367.191486
2740  412775.379384  414600   1824.620616
1820  849590.750881  856700   7109.249119
2248  890752.613598  879600 -11152.613598
4093  696737.727366  723100  26362.272634
5621  438688.443518  409000 -29688.443518
435   670676.695975  692900  22223.304025
5537  476482.467320  394700 -81782.467320
5238  479191.173293  472600  -6591.173293
5434  343064.771666  341600  -1464.771666
1736  485833.148052  446200 -39633.148052

Regression statistics

                      Mean Error (ME) : -918.7462
       Root Mean Squared Error (RMSE) : 5986

# Backward Elimination

In [ ]:
def train_model(variables):
    model=LinearRegression()
    model.fit(train_X[variables],train_y)
    return model

def score_model (model, variables):
    return AIC_score (train_y, model.predict(train_X[variables]),model)

best_model, best_variables=backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: LAND_SF, GROSS_AREA, GROSS_TAX, YR_BUILT, YR_REMODEL, BED_RMS, FULL_BTH, HLF_BTH, KITCHENS, TT_RMS, FIREPLACES, NUM_PARKING
Start: score=54547.07
Step: score=54545.07, remove NUM_PARKING
Step: score=54543.07, remove KITCHENS
Step: score=54541.78, remove LAND_SF
Step: score=54540.44, remove BED_RMS
Step: score=54540.28, remove FIREPLACES
Step: score=54540.28, remove None
['GROSS_AREA', 'GROSS_TAX', 'YR_BUILT', 'YR_REMODEL', 'FULL_BTH', 'HLF_BTH', 'TT_RMS']


In [ ]:
regressionSummary(valid_y, best_model.predict(valid_X[best_variables]))


Regression statistics

                      Mean Error (ME) : -1042.0621
       Root Mean Squared Error (RMSE) : 59738.6808
            Mean Absolute Error (MAE) : 28177.1382
          Mean Percentage Error (MPE) : -1.4576
Mean Absolute Percentage Error (MAPE) : 6.1831


# Forward Selection Model

In [ ]:
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: LAND_SF, GROSS_AREA, GROSS_TAX, YR_BUILT, YR_REMODEL, BED_RMS, FULL_BTH, HLF_BTH, KITCHENS, TT_RMS, FIREPLACES, NUM_PARKING
Start: score=59333.85, constant
Step: score=54769.35, add GROSS_TAX
Step: score=54620.53, add GROSS_AREA
Step: score=54576.58, add FULL_BTH
Step: score=54555.90, add HLF_BTH
Step: score=54546.26, add YR_REMODEL
Step: score=54543.74, add YR_BUILT
Step: score=54540.28, add TT_RMS
Step: score=54540.28, add None
['GROSS_TAX', 'GROSS_AREA', 'FULL_BTH', 'HLF_BTH', 'YR_REMODEL', 'YR_BUILT', 'TT_RMS']


# Regression Output

In [ ]:
train_df=train_X.join(train_y)

predictors= train_X.columns
formula='TOTAL_VALUE~'+'+'.join(predictors)

reg=sm.ols(formula=formula,data=train_df).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:            TOTAL_VALUE   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     1492.
Date:                Mon, 30 Oct 2023   Prob (F-statistic):               0.00
Time:                        13:14:48   Log-Likelihood:                -27260.
No. Observations:                2149   AIC:                         5.455e+04
Df Residuals:                    2136   BIC:                         5.462e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -2.068e+05   8.73e+04     -2.369      